[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap09/cap09-main.ipynb)

# Capítulo 9 - Introdução ao Policy Gradient - Reinforce


## Configurações Iniciais

In [ ]:
import sys
from IPython.display import clear_output

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gymnasium[box2d]

    # para salvar videos
    !apt-get install -y ffmpeg xvfb x11-utils

    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [ ]:
if IN_COLAB:
    import os
    os.system("Xvfb :1 -screen 0 1400x900x24 &")
    os.environ['DISPLAY'] = ':1'

In [ ]:
import gymnasium as gym
import numpy as np

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path

from cap08.models_torch_pg import PolicyModelPG, test_policy
from cap08.reinforce import run_reinforce
from cap08.reinforce_advantage import run_reinforce_with_adv

## 1 - Rodando o Reinforce (Vanilla Policy Gradient)

In [ ]:
ENV_NAME, rmax = "CartPole-v1", 500
#ENV_NAME, rmax = "Acrobot-v1", 0
#ENV_NAME, rmax = "LunarLander-v2", 150
#ENV_NAME, rmax = "MountainCar-v0", -20

EPISODES = 1000
GAMMA    = 0.99

env = gym.make(ENV_NAME)
inputs = env.observation_space.shape[0]
outputs = env.action_space.n
policy1 = PolicyModelPG(inputs, [128, 512], outputs, lr=0.0005)

returns1, policy1 = run_reinforce(env, EPISODES, 0.95, initial_policy=policy1)
clear_output()

print("Últimos episódios do treinamento: media =", np.mean(returns1[-20:]), ", desvio padrao =", np.std(returns1[-20:]))


In [ ]:
# Exibe um gráfico episódios x retornos (não descontados)
plot_result(returns1, rmax, window=50)

In [ ]:
# Roda alguns episódigos com o modelo e salva os vídeos em arquivos
# TODO: 
#  - rever, deixar parecido com a forma da função usada nos notebooks do stable-baselines, gravando um só vídeo de tamanho fixo (com possivelmente vários episódios)
#  - permitir renderizar OU gravar -- passar apenas a string do ambiente e chamar o make appropriado

env1 = gym.make(ENV_NAME, render_mode="rgb_array")
video_env = gym.wrappers.RecordVideo(env1, "./vid-reinf", episode_trigger=(lambda ep : True), video_length=1_500)
test_policy(video_env, policy1, False, 3, render=False)
video_env.close()

In [ ]:
display_videos_from_path('./vid-reinf')

## 2 - Rodando o Reinforce com Advantage

In [ ]:
ENV_NAME, rmax = "CartPole-v1", 500
#ENV_NAME, rmax = "Acrobot-v1", 0
#ENV_NAME, rmax = "LunarLander-v2", 150
#ENV_NAME, rmax = "MountainCar-v0", -20

EPISODES = 1000
GAMMA    = 0.99

env = gym.make(ENV_NAME)
inputs = env.observation_space.shape[0]
outputs = env.action_space.n
policy2 = PolicyModelPG(inputs, [128, 512], outputs, lr=0.001)

returns2, policy2 = run_reinforce_with_adv(env, EPISODES, GAMMA, initial_policy=policy2)
clear_output()

print("Últimos episódios do treinamento: media =", np.mean(returns2[-20:]), ", desvio padrao =", np.std(returns2[-20:]))

In [ ]:
# Exibe um gráfico episódios x retornos (não descontados)
plot_result(returns2, rmax, window=50)

In [ ]:
# Roda alguns episódigos com o modelo e salva os vídeos em arquivos
env2 = gym.make(ENV_NAME)
env1 = gym.make(ENV_NAME, render_mode="rgb_array")
video_env = gym.wrappers.RecordVideo(env1, "./vid-reinf-adv", episode_trigger=(lambda ep : True), video_length=1_600)
test_policy(video_env, policy2, False, 3, render=False)
video_env.close()

In [ ]:
display_videos_from_path('./vid-reinf-adv')

## 3 - Experimentos


### 3.1 Comparações Básicas


In [ ]:
AUTO_LOAD = False

In [ ]:
env = gym.make("CartPole-v1")
NUM_EPISODES = 2000

results3 = []

results3.append( repeated_exec(3, f"Reinforce", run_reinforce, env, NUM_EPISODES, gamma=0.95, auto_load=AUTO_LOAD) )
clear_output()

results3.append( repeated_exec(3, f"Reinforce-Adv", run_reinforce_with_adv, env, NUM_EPISODES, gamma=0.95, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
#plot_multiple_results(results3, cumulative=False, x_log_scale=False)

In [ ]:
plot_multiple_results(results3, cumulative=False, plot_stddev=True)

### 3.2 Variando Parâmetros da Rede Neural

Vamos mostrar comparações variando apenas a quantidade de neurônios da (única) camada intermediária.

Você pode também variar todos os outros parâmetros da rede neural: quantidade de camadas e taxa de aprendizagem.

In [ ]:
env = gym.make("CartPole-v1")

NUM_EPISODES = 1000

print("Observation Space - ", env.observation_space.shape)
print("Observation Space - shape", env.observation_space.shape)
print("Action Space - ", env.action_space)
print("Action Space - number of actions - ", env.action_space.n)

In [ ]:
results4 = []

for hidden_layer_size in [16, 128]:
    policy_model = PolicyModelPG(env.observation_space.shape[0], [hidden_layer_size], env.action_space.n, lr=0.01)
    results4.append( repeated_exec(3, f"Reinforce (hnodes={hidden_layer_size})", run_reinforce, env, NUM_EPISODES, 0.99, policy_model, auto_load=AUTO_LOAD) )
    clear_output()
    results4.append( repeated_exec(3, f"Reinforce-Adv (hnodes={hidden_layer_size})", run_reinforce_with_adv, env, NUM_EPISODES, 0.99, policy_model, auto_load=AUTO_LOAD) )
    clear_output()

In [ ]:
#results4

In [ ]:
plot_multiple_results(results4, cumulative=False)

In [ ]:
#plot_multiple_results(results4, cumulative=False, plot_stddev=True)